In [2]:
import copy
import numpy as np
import os
import matplotlib.pyplot as plt
import pprint

from copy import deepcopy
from evo.core import metrics, sync
from evo.core.trajectory import PosePath3D
from evo.tools import plot

from tools.tum_tools import *
from tools.tum_plots import *
from tools.tum_definitions import *

%load_ext autoreload
%autoreload 2

%matplotlib inline
%matplotlib widget


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
!evo_traj tum ekf_pose_with_covariance.tum --ref in_pose_with_covariance.tum -p

In [ ]:
# Load previously computed data
import pickle

data_root = ""

gt_file = os.path.join("data/full", "gt_poses.pkl")
traj_file = os.path.join("data/full2", "kiss_trajectories_corrected_corr.pkl")

with open(gt_file, 'rb') as f:
    gt_dict = pickle.load(f)
    
with open(traj_file, 'rb') as f:
    traj_dict = pickle.load(f)
    
    

In [ ]:
# Extract relevant data
current_key = "mdv3_full_no_merge"
traj_ref = gt_dict['trajectory']
traj_est = traj_dict[current_key][2]



In [ ]:
traj_ref.timestamps

In [ ]:
len(traj_est.timestamps)

In [ ]:
# Align data
max_diff = 1

traj_ref, traj_est = sync.associate_trajectories(traj_ref, traj_est, max_diff)


In [ ]:
# Compute Metrics
ape_metric = metrics.APE()
ape_metric.process_data((traj_ref, traj_est))
ape_stats = ape_metric.get_all_statistics()
pprint.pprint(ape_stats)

In [ ]:
plot_ape_errors(traj_ref, traj_est, ape_metric, ape_stats)
plt.savefig(f"figures/ape_{current_key}.pdf", format="pdf", bbox_inches="tight")

In [ ]:
# PLOT THE FULL SET

max_diff = 0.01
results = {}

for recording in Mdv3Dataset:
    if traj_dict.get(recording.value) is None:
        print(f"Info: Did not found a recorded trajectory for {recording.value}")
        continue

    traj_ref = gt_dict['trajectory']
    traj_est = traj_dict[recording.value][2]
    traj_ref, traj_est = sync.associate_trajectories(traj_ref, traj_est, max_diff)
    
    ape_metric = metrics.APE()
    ape_metric.process_data((traj_ref, traj_est))
    ape_stats = ape_metric.get_all_statistics()
    plot_ape_errors(traj_ref, traj_est, ape_metric, ape_stats, title=recording.value)
    plt.savefig(f"figures/ape_{recording.value}.pdf", format="pdf", bbox_inches="tight")
    results[recording.value] = ape_stats
    

In [ ]:
for recording, stats in results.items():
    print(f"Statistics for {recording}")
    pprint.pprint(stats)

In [ ]:
results

In [ ]:
import csv
import itertools

fields = [ 'Type', 'max', 'mean', 'median', 'min', 'rmse', 'sse', 'std']


with open("test_output.csv", "w") as f:
    w = csv.DictWriter(f, fields)
    w.writeheader()
    for k in results:
        w.writerow({field: results[k].get(field) or k for field in fields})

In [ ]:
plot_trajectories_from_poses(traj_ref, traj_est)